# import libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time
import numpy as np
from collections import Counter
from keras.models import load_model
from keras.datasets import cifar10

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
drive_path = '/content/drive/MyDrive/UTS_MKTK'

Mounted at /content/drive/


# read dataset: CIFAR-10

In [3]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

CLASS_NAMES= ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

170508288/170498071 [==============================] - 2s 0us/step


filter only airplane, automobile, ship, and truck. each category has 800 data

In [4]:
for i in [0, 1, 8, 9]:
  idx = (train_labels == i).reshape(train_images.shape[0])
  train_img = train_images[idx][:500]
  val_img = train_images[idx][500:750]
  test_img = train_images[idx][750:800]
  if i == 0:
    airplane_img_tr = train_img
    airplane_img_val = val_img
    airplane_img_te = test_img    
  elif i == 1:
    automobile_img_tr = train_img
    automobile_img_val = val_img
    automobile_img_te = test_img
  elif i == 8:
    ship_img_tr = train_img
    ship_img_val = val_img
    ship_img_te = test_img
  else:
    truck_img_tr = train_img
    truck_img_val = val_img
    truck_img_te = test_img

training_image = np.concatenate((airplane_img_tr, automobile_img_tr, ship_img_tr, truck_img_tr), axis=0)
validation_image = np.concatenate((airplane_img_val, automobile_img_val, ship_img_val, truck_img_val), axis=0)
testing_image = np.concatenate((airplane_img_te, automobile_img_te, ship_img_te, truck_img_te), axis=0)

In [5]:
A = []
B = []
C = []
D = []
for i in range(500):
  A.append([0])
  B.append([1])
  C.append([2])
  D.append([3])

A = np.array(A)
B = np.array(B)
C = np.array(C)
D = np.array(D)

training_label = np.concatenate((A, B, C, D), axis=0)

In [6]:
A = []
B = []
C = []
D = []
for i in range(250):
  A.append([0])
  B.append([1])
  C.append([2])
  D.append([3])

A = np.array(A)
B = np.array(B)
C = np.array(C)
D = np.array(D)

validation_label = np.concatenate((A, B, C, D), axis=0)

In [7]:
A = []
B = []
C = []
D = []
for i in range(50):
  A.append([0])
  B.append([1])
  C.append([2])
  D.append([3])

A = np.array(A)
B = np.array(B)
C = np.array(C)
D = np.array(D)

testing_label = np.concatenate((A, B, C, D), axis=0)

In [8]:
print(training_image.shape)
print(validation_image.shape)
print(testing_image.shape)

print(training_label.shape)
print(validation_label.shape)
print(testing_label.shape)

class_names = ['airplane', 'automobile', 'ship', 'truck']

(2000, 32, 32, 3)
(1000, 32, 32, 3)
(200, 32, 32, 3)
(2000, 1)
(1000, 1)
(200, 1)


In [9]:
train_ds = tf.data.Dataset.from_tensor_slices((training_image, training_label))
test_ds = tf.data.Dataset.from_tensor_slices((testing_image, testing_label))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_image, validation_label))

# define Preprocessing

In [10]:
def process_images(image, label):
    image = tf.image.per_image_standardization(image) #strandarisasi
    image = tf.image.resize(image, (224,224))         #resize
    return image, label

In [11]:
train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()
validation_ds_size = tf.data.experimental.cardinality(validation_ds).numpy()
print("Training data size:", train_ds_size)
print("Test data size:", test_ds_size)
print("Validation data size:", validation_ds_size)

Training data size: 2000
Test data size: 200
Validation data size: 1000


In [12]:
train_ds = (train_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=10, drop_remainder=True))
test_ds = (test_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=10, drop_remainder=True))
validation_ds = (validation_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=10, drop_remainder=True))

# model

In [13]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense
from tensorflow.keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU, concatenate
import tensorflow.keras.backend as K
# Creating Densenet121
def densenet(input_shape, n_classes, filters = 32):
    
    #batch norm + relu + conv
    def bn_rl_conv(x,filters,kernel=1,strides=1):
        
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(filters, kernel, strides=strides,padding = 'same')(x)
        return x
    
    def dense_block(x, repetition):
        
        for _ in range(repetition):
            y = bn_rl_conv(x, 4*filters)
            y = bn_rl_conv(y, filters, 3)
            x = concatenate([y,x])
        return x
        
    def transition_layer(x):
        
        x = bn_rl_conv(x, K.int_shape(x)[-1] //2 )
        x = AvgPool2D(2, strides = 2, padding = 'same')(x)
        return x
    
    input = Input (input_shape)
    x = Conv2D(64, 7, strides = 2, padding = 'same')(input)
    x = MaxPool2D(3, strides = 2, padding = 'same')(x)
    
    for repetition in [6,12,24,16]:
        
        d = dense_block(x, repetition)
        x = transition_layer(d)
    x = GlobalAveragePooling2D()(d)
    output = Dense(n_classes, activation = 'softmax')(x)
    
    model = Model(input, output)
    return model
input_shape = 224, 224, 3
n_classes = 4
model = densenet(input_shape,n_classes)

In [15]:
root_logdir = os.path.join(os.curdir, "logs\\fit\\")
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()

#Defining callbacks to get the best model possible
my_calls = [keras.callbacks.EarlyStopping(monitor='val_loss',patience=30),
            keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=10, min_lr=1.5e-5),
            keras.callbacks.ModelCheckpoint(drive_path + "/Model2.h5",verbose=1,save_best_only=True),
            keras.callbacks.TensorBoard(run_logdir)]

In [16]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.Adam(lr=0.01), metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 64  9472        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 56, 56, 64)   0           ['conv2d[0][0]']                 
                                                                                              

In [17]:
hist = model.fit(train_ds, epochs=100, validation_data=validation_ds,
                 validation_freq=1, callbacks=my_calls)

Epoch 1/100
200/200 [==============================] - ETA: 0s - loss: 2.7784 - accuracy: 0.3300
Epoch 00001: val_loss improved from inf to 6.40465, saving model to /content/drive/MyDrive/UTS_MKTK/Model2.h5
200/200 [==============================] - 132s 418ms/step - loss: 2.7784 - accuracy: 0.3300 - val_loss: 6.4046 - val_accuracy: 0.2810 - lr: 0.0100
Epoch 2/100
200/200 [==============================] - ETA: 0s - loss: 1.3078 - accuracy: 0.3645
Epoch 00002: val_loss improved from 6.40465 to 1.25990, saving model to /content/drive/MyDrive/UTS_MKTK/Model2.h5
200/200 [==============================] - 79s 394ms/step - loss: 1.3078 - accuracy: 0.3645 - val_loss: 1.2599 - val_accuracy: 0.3810 - lr: 0.0100
Epoch 3/100
200/200 [==============================] - ETA: 0s - loss: 1.5407 - accuracy: 0.3660
Epoch 00003: val_loss did not improve from 1.25990
200/200 [==============================] - 74s 368ms/step - loss: 1.5407 - accuracy: 0.3660 - val_loss: 598058.5625 - val_accuracy: 0.2500 

KeyboardInterrupt: ignored

# load model

In [18]:
model = load_model(drive_path + "/Model2.h5")

In [19]:
model.evaluate(test_ds)

20/20 [==============================] - 5s 102ms/step - loss: 0.6758 - accuracy: 0.7550


[0.6757926344871521, 0.7549999952316284]